In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)
    
import utils
import api
from api.gui.pairs import tab_ as pairs_tab
from api.gui.details import tab_ as details_tab
from api.gui.regression import tab_ as regression_tab
from api.gui.base import MultiTab, Tab
from api.data.base import tables, data_path
from api.data.processing import get_current_data

In [3]:
data = get_current_data()

In [5]:
cds_ref = pd.read_csv(data_path / 'cds_reference.csv').query('exists == True').drop_duplicates(subset='security')

In [4]:
cds_ref = pd.read_csv(data_path / 'cds_reference.csv').query('exists == True').drop_duplicates(subset='security')
cds_mkt = pd.read_csv(data_path / 'cds_market.csv').drop_duplicates(subset='security')

In [9]:
cds_ref = pd.read_csv(data_path / 'cds_reference.csv').query('exists == True').drop_duplicates(subset='security')
cds_mkt = pd.read_csv(data_path / 'cds_market.csv').drop_duplicates(subset='security')

name_mapping = {
    'security': 'cds_name',
    'last_price': 'cds_spread',
    'years': 'cds_years',
    'capital_stack': 'cds_capital_stack'
}


cds_data = cds_mkt.merge(cds_ref.set_index('security'), left_on='security', right_index=True, how='left')\
    .rename(name_mapping, axis=1)\
    .drop(['exists', 'ticker', 'cds_ticker_5y'], axis=1)\
    .dropna(subset='cds_spread')

In [33]:
issuer = 'Asdasda'
currency = 'EUR'

In [16]:
mask = (cds_data['issuer_equity_ticker'] == issuer)
cds_data_ = cds_data.loc[mask]

In [15]:
from api.data.utils import _capital_stack as capital_stack_items 

In [14]:
cds_data_

cds_name  cds_spread currency  cds_years  \
576    ISPIM SPA CDS EUR SR 2Y D14 Curncy        66.0      EUR          2   
577   ISPIM SPA CDS EUR SUB 2Y D14 Curncy       197.0      EUR          2   
578   ISPIM SPA CDS EUR SLA 2Y D14 Curncy       102.5      EUR          2   
579    ISPIM SPA CDS EUR SR 5Y D14 Curncy       115.0      EUR          5   
580   ISPIM SPA CDS EUR SUB 5Y D14 Curncy       315.5      EUR          5   
581   ISPIM SPA CDS EUR SLA 5Y D14 Curncy       185.0      EUR          5   
582    ISPIM SPA CDS EUR SR 7Y D14 Curncy       133.0      EUR          7   
583   ISPIM SPA CDS EUR SUB 7Y D14 Curncy       355.0      EUR          7   
584   ISPIM SPA CDS EUR SLA 7Y D14 Curncy       214.0      EUR          7   
585   ISPIM SPA CDS EUR SR 10Y D14 Curncy       147.0      EUR         10   
586  ISPIM SPA CDS EUR SUB 10Y D14 Curncy       385.0      EUR         10   
587  ISPIM SPA CDS EUR SLA 10Y D14 Curncy       235.5      EUR         10   

    cds_capital_stack issuer_equity_ticker  
576                SR        ISP IM Equity  
577               SUB        ISP IM Equity  
578               SLA        ISP IM Equity  
579                SR        ISP IM Equity  
580               SUB        ISP IM Equity  
581               SLA        ISP IM Equity  
582                SR        ISP IM Equity  
583               SUB        ISP IM Equity  
584               SLA        ISP IM Equity  
585                SR        ISP IM Equity  
586               SUB        ISP IM Equity  
587               SLA        ISP IM Equity

In [34]:
mask = (cds_data['issuer_equity_ticker'] == issuer)
cds_data_ = cds_data.loc[mask]

In [36]:
cds_data_

Empty DataFrame
Columns: [cds_name, cds_spread, currency, cds_years, cds_capital_stack, issuer_equity_ticker]
Index: []

In [25]:
res = {}
for cs in capital_stack_items:
    data__ = cds_data_.query(f"cds_capital_stack == '{cs}' and currency == '{currency}'")[['cds_years', 'cds_spread']]
    res[cs] = data__.groupby('cds_years').mean()

In [23]:
data__.groupby('cds_years').mean()

cds_spread
cds_years            
2                66.0
5               115.0
7               133.0
10              147.0

In [37]:
mask = (cds_data['issuer_equity_ticker'] == issuer)
cds_data_ = cds_data.loc[mask]
    
if not cds_data_.empty:
    res = {}
    for cs in capital_stack_items:
        data__ = cds_data_.query(f"cds_capital_stack == '{cs}' and currency == '{currency}'")[['cds_years', 'cds_spread']]
        res[cs] = data__.groupby('cds_years').mean()


    pairs = [
        ('SLA', 'SR'),
        ('SUB', 'SLA'),
    ]

    ratios = {}
    for pair in pairs:
        cs1, cs2 = pair
        ratios[pair] = (res[cs1] / res[cs2]).squeeze().rename(' / '.join(pair))
    all_ratios = pd.concat(ratios.values(), axis=1)

In [38]:
all_ratios

SLA / SR  SUB / SLA
cds_years                     
2          1.553030   1.921951
5          1.608696   1.705405
7          1.609023   1.658879
10         1.602041   1.634820

In [ ]:
data['cds_ticker_5y']

In [ ]:
t = tables.bonds_reference.query(f"issuer_equity_ticker =='ISP IM Equity'")

In [ ]:
pivot = pivot_ui(
    data.reset_index(),
    height=1000,
    # pivot_type='pivot'
    rows=['country', 'ticker'],
    cols=['maturity_bucket']
)
out = w.Output()
with out:
    display(pivot)
pivot_tab = Tab(out, name='pivot')

In [ ]:
app = MultiTab(
    tabs=[
        pairs_tab,
        regression_tab,
        details_tab,
        pivot_tab
    ]
)

### Credit Monitor

In [ ]:
cols = [
    'ticker', 
    'name', 
    'currency', 
    'country', 
    'bics_sector',
    'capital_stack', 
    'maturity_bucket',
    'rating_bucket', 
    'area',
    'last_price', 
    'yield_to_call',
    'ytm_flat',
    'ytm_fwd', 
    'xccy_usd', 
    'i_spread', 
    'z_spread', 
    'aws_spread'
]

In [ ]:
currency_items = data['currency'].drop_duplicates().to_list()
capital_stack_items = data['capital_stack'].drop_duplicates().to_list()
items = [
    'yield_to_call',
    'ytm_flat',
    'ytm_fwd', 
    'xccy_usd', 
    'i_spread', 
    'z_spread', 
    'aws_spread'
]

In [ ]:
from api.gui.utils import make_data_table, custom_sort

issuer_sel = v.Autocomplete(
    v_model=None,
    items=data['ticker'].drop_duplicates().to_list(),
    label='issuer',
    dense=True,
    outlined=True
)

curr_sel = v.Autocomplete(
    v_model=None,
    items=currency_items,
    label='currency',
    dense=True,
    outlined=True
)

cap_stack_sel = v.Autocomplete(
    v_model=None,
    items=capital_stack_items,
    label='capital stack',
    dense=True,
    outlined=True
)

spread_sel = v.Autocomplete(
    v_model=None,
    items=items,
    label='field',
    dense=True,
    outlined=True
)



relevant_fields = [
    'ticker',
    'country',
    'currency',
    'maturity_bucket',
    'rating_bucket',
]


table_btn = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-table']),
        'Table',
    ],
)

out = w.Output()


param_box = v.Container(
    children=[
        v.Row(
            children=[
                
                v.Col(
                    cols=2,
                    children=[issuer_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[curr_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[cap_stack_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[spread_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[table_btn],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        ),
    ]
)

tab = w.VBox(
    children=[
        param_box,
        out,
    ]
)


def on_click_show_table(widget, event, payload):
    with out:
        out.clear_output()
        issuer = issuer_sel.v_model
        currency = curr_sel.v_model
        data_ = data.query(f"ticker == '{issuer}' and currency == '{currency}'").sort_values('capital_stack')[['name', 'maturity_bucket', 'rating_bucket', 'capital_stack', 'currency', *items]]

        table1 = make_data_table(data_, items_per_page=100)

        
        display(table1.box)
        
        item = spread_sel.v_model
        currency = curr_sel.v_model
        res = {}

        for cs in capital_stack_items:
            data__ = data_.query(f"capital_stack == '{cs}' and currency == '{currency}'")[['maturity_bucket', item]]
            res[cs] = data__.groupby('maturity_bucket').mean()

        pairs = [
            ('at1', 'lt2'),
            ('lt2', 'snp'),
            ('snp', 'sp')
        ]

        ratios = {}
        for pair in pairs:
            cs1, cs2 = pair
            ratios[pair] = (res[cs1] / res[cs2]).squeeze().rename(' / '.join(pair))
        all_ratios = pd.concat(ratios.values(), axis=1)
        table2 = make_data_table(custom_sort(all_ratios, axis=0))

    
        display(table2.box)
    
    
table_btn.on_event('click', on_click_show_table) 

In [ ]:
tab

In [ ]:
out

In [ ]:
def get_col():
    return [
        *relevant_fields,
        spread_sel.v_model,
    ]

def get_mat():
    return [
        '2yr',
        '5yr',
        '7yr',
        '10yr',
        '15yr'
    ]


def get_rtg():
    return [
        'AA',
        'A',
        'BBB',
        'BB',
        'B'
    ]

In [ ]:
data.loc[data['ytm_fwd'].between(6.5, 7)][['name', 'maturity_bucket', 'rating_bucket', 'capital_stack', 'currency', *items]].query('currency == "USD"').to_clipboard()

In [ ]:
issuer = issuer_sel.v_model
currency = curr_sel.v_model

data_ = data.query(f"ticker == '{issuer}' and currency == '{currency}'").sort_values('capital_stack')[['name', 'maturity_bucket', 'rating_bucket', 'capital_stack', 'currency', *items]]

In [ ]:
capital_stack_items
item = spread_sel.v_model


currency = curr_sel.v_model

res = {}

for cs in capital_stack_items:
    data__ = data_.query(f"capital_stack == '{cs}' and currency == '{currency}'")[['maturity_bucket', item]]
    res[cs] = data__.groupby('maturity_bucket').mean()

In [ ]:
pairs = [
    ('at1', 'lt2'),
    ('lt2', 'snp'),
    ('snp', 'sp')
]

In [ ]:
ratios = {}
for pair in pairs:
    cs1, cs2 = pair
    ratios[pair] = (res[cs1] / res[cs2]).squeeze().rename(' / '.join(pair))

In [ ]:
all_ratios = pd.concat(ratios.values(), axis=1)

In [ ]:
from api.gui.utils import get_str_size, _get_width, get_columns_width

In [ ]:
get_columns_width(data.reset_index())

In [ ]:
series = data['coupon'].astype(str)

In [ ]:
max_name = get_str_size((series.name or '') + ' ', 9, 'calibri.ttf')[0]
max_value = get_str_size(series[series.str.len().idxmax()], 12, 'calibri.ttf')[0]
max_all = max(max_name, max_value)

In [ ]:
from api.data.base import tables

In [ ]:
DataGrid(data, auto_fit_columns=True, auto_fit_params={'area': 'body'})

In [ ]:
for col in all_ratios:
    break

In [ ]:
col

In [ ]:
custom_sort(all_ratios, axis=0)

In [ ]:
table = make_data_table(custom_sort(all_ratios, axis=0))

In [ ]:
table.box

In [ ]:
(res[cs1] / res[cs2]).squeeze().rename(' / '.join(pair))

In [ ]:
res[cs2]

In [ ]:
res[cs1]